# Improvement Trial

Previously I forgot to manage memory properly, but this time I will also look at the score each player got in each game

In [1]:
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np

Using TensorFlow backend.


In [2]:
#game is now using lag=2 instead of lag=8 for q-score
#maybe this will help guide it to higher point values
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 2
test_config['discount'] = 1.

init_game = Game(n_players=4, q_lag=2, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []

for temperature in [1., 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.01, 0.001, 0.]:
    print 'TEMPERATURE: %.03f' % temperature
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    for cycle in range(3):
        print 'in cycle %d' % cycle
        for i in range(35):
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config)
            game.run()
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % np.mean(player_wins[i])/105
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

TEMPERATURE: 1.000
in cycle 0
in cycle 1


ai.py:361: RuntimeWarning: invalid value encountered in divide
  return selected/np.sum(selected)
ai.py:334: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


in cycle 2
WINNING SCORE AVERAGE: -12.2
---PLAYER 0 STATS---
WIN AVERAGE: 34.000
SCORE AVERAGE: -30.2
---PLAYER 1 STATS---
WIN AVERAGE: 25.000
SCORE AVERAGE: -29.8
---PLAYER 2 STATS---
WIN AVERAGE: 24.000
SCORE AVERAGE: -30.8
---PLAYER 3 STATS---
WIN AVERAGE: 23.000
SCORE AVERAGE: -30.0
TEMPERATURE: 0.900
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: -10.4
---PLAYER 0 STATS---
WIN AVERAGE: 30.000
SCORE AVERAGE: -28.8
---PLAYER 1 STATS---
WIN AVERAGE: 28.000
SCORE AVERAGE: -28.4
---PLAYER 2 STATS---
WIN AVERAGE: 27.000
SCORE AVERAGE: -28.3
---PLAYER 3 STATS---
WIN AVERAGE: 22.000
SCORE AVERAGE: -35.2
TEMPERATURE: 0.800
in cycle 0
in cycle 1
in cycle 2
WINNING SCORE AVERAGE: -11.6
---PLAYER 0 STATS---
WIN AVERAGE: 27.000
SCORE AVERAGE: -32.3
---PLAYER 1 STATS---
WIN AVERAGE: 20.000
SCORE AVERAGE: -31.0
---PLAYER 2 STATS---
WIN AVERAGE: 27.000
SCORE AVERAGE: -29.7
---PLAYER 3 STATS---
WIN AVERAGE: 32.000
SCORE AVERAGE: -25.7
TEMPERATURE: 0.700
in cycle 0
in cycle 1
in cycle 2
WI

ai.py:360: RuntimeWarning: overflow encountered in power
  selected = x**(1/temperature)


in cycle 1
in cycle 2
WINNING SCORE AVERAGE: 4.7
---PLAYER 0 STATS---
WIN AVERAGE: 29.000
SCORE AVERAGE: -7.7
---PLAYER 1 STATS---
WIN AVERAGE: 25.000
SCORE AVERAGE: -9.4
---PLAYER 2 STATS---
WIN AVERAGE: 27.000
SCORE AVERAGE: -8.2
---PLAYER 3 STATS---
WIN AVERAGE: 27.000
SCORE AVERAGE: -8.0
TEMPERATURE: 0.000
in cycle 0


ValueError: probabilities do not sum to 1

going to add epsilon terms to some numpy values to avoid these issues

In [3]:
i

0

In [4]:
game.turn

63

In [5]:
np.mean(overall_winning_scores,1)

array([-12.16190476, -10.4       , -11.59047619, -10.28571429,
        -9.55238095,  -6.32380952,  -7.61904762,  -8.8952381 ,
        -7.91428571,  -9.63809524,  -9.38095238,  -7.78095238,   4.66666667])

In [15]:
np.mean(overall_player_scores, ( 2,))

array([[-30.16190476, -29.82857143, -30.75238095, -30.03809524],
       [-28.77142857, -28.38095238, -28.3047619 , -35.2       ],
       [-32.33333333, -31.03809524, -29.65714286, -25.73333333],
       [-34.39047619, -32.86666667, -30.12380952, -21.84761905],
       [-38.95238095, -24.87619048, -27.11428571, -23.14285714],
       [-45.82857143, -20.32380952, -26.18095238, -20.32380952],
       [-47.12380952, -19.8       , -27.72380952, -28.18095238],
       [-46.4952381 , -26.6952381 , -22.60952381, -22.96190476],
       [-49.28571429, -24.96190476, -28.95238095, -20.6       ],
       [-48.27619048, -28.44761905, -26.45714286, -24.79047619],
       [-41.39047619, -25.44761905, -29.19047619, -28.97142857],
       [-41.52380952, -31.23809524, -28.65714286, -22.46666667],
       [ -7.72380952,  -9.36190476,  -8.18095238,  -8.02857143]])

In [11]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test003_%%s.h5' % i)

not sure why all player scores improved for the last row...perhaps training should lower temperature more quickly